# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="/Users/aristizabal-factored/Documents/Value-based-methods/p1_navigation/Banana.app")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
from unityagents import UnityEnvironment
from dqn_agent import DQNAgent
from experience_replay import MemoryBuffer, PriorityBuffer
from value_estimators import VanillaEstimator, DoubleDQNEstimator
from networks import VanillaQNetwork, DuelingQNetwork
import numpy as np

def memory_config_generator(local_model, target_model, gamma=0.99):
	return {
    "max_size": int(1e5),
	"local_model": local_model,
	"target_model": target_model,
	"estimator": estimator,
	"gamma": gamma,
	"epsilon": 1e-5,
	"priority_amount": 0.2,
}

# Initialize the environment
env = UnityEnvironment(file_name="/Users/aristizabal-factored/Documents/Value-based-methods/p1_navigation/Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

env_info = env.reset(train_mode=True)[brain_name]
state_size = len(env_info.vector_observations[0])
action_size = brain.vector_action_space_size

agents_dict = {
	"VanillaDQN": {
		"network": VanillaQNetwork,
		"estimator": VanillaEstimator,
		"memory": MemoryBuffer,
	},
	"DuelingDQN": {
		"network": DuelingQNetwork,
		"estimator": VanillaEstimator,
		"memory": MemoryBuffer,
	},
	"DoubleDQN": {
		"network": VanillaQNetwork,
		"estimator": DoubleDQNEstimator,
		"memory": MemoryBuffer,
	 },
	"PrioritySweepDQN": {
		"network": VanillaQNetwork,
		"estimator": VanillaEstimator,
		"memory": PriorityBuffer,
	},
	"RainbowDQN": {
		"network": DuelingQNetwork,
		"estimator": DoubleDQNEstimator,
		"memory": PriorityBuffer,
	}
}

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## Train and compare
Having configured the agents, we can run the experiment for each of them and obtain some information about their performance compared to each other.

In [ ]:
gamma = 0.99
hidden_layers = [128, 128, 64, 64, 64, 32, 32]

for agent_name, agent_config in agents_dict.items():
	Network = agent_config["network"]
	Estimator = agent_config["estimator"]
	Memory = agent_config["memory"]

	local_model = Network(state_size, action_size, hidden_layers=hidden_layers)
	target_model = Network(state_size, action_size, hidden_layers=hidden_layers)

	estimator = Estimator()

	memory_config = memory_config_generator(local_model, target_model)
	memory = Memory(memory_config)
	agent = DQNAgent(env, estimator, local_model, target_model, memory)
	print("Running experiment with agent {}".format(agent_name))
	agent.train(episodes=1250, epsilon=0.2, max_steps=int(1e8), lr=1e-4, alpha=0.001)
	print("Saving agent and rewards")
	model_path = "checkpoints/{}.pth".format(agent_name)
	rewards_path = "checkpoints/{}_rewards.npz".format(agent_name)
	agent.save(model_path)
	rewards = np.array(agent.rewards)
	np.savez(rewards_path, rewards)

## Load and plot
Having trained all the agents, we can observe how they did during train time. We are interested in seing improvements in convergence time

In [4]:
rewards_dict = {}

for agent in agents_dict.keys():
	rewards = np.load(f"checkpoints/{agent}_rewards.npz")
	rewards_dict[agent] = rewards

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/PriorySweepDQN_rewards.npz'